<a href="https://colab.research.google.com/github/mateusfarah/Hackathon_01-FCCD/blob/main/Submission_kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Submission  -                Hackaton 01 FCCD    **

---



Este arquivo apenas repete o processo do projeto mas preparado para os dados de teste e envia as predições para o kaggle.



---
O notebook está dividido em:



1.   Importação das bibliotecas
2.   Separação dos dados de treino e dados de teste

1.   Feature Engineering / Processamento dos dados
2.   Desenvolvimento dos modelos

1.   Métricas de validação






In [55]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


- **Importando as bibliotecas**

In [56]:
!pip install category_encoders

In [57]:
pip install optuna

In [58]:
from category_encoders import TargetEncoder
import numpy as np
import sqlite3
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score ,f1_score ,make_scorer,confusion_matrix, ConfusionMatrixDisplay,roc_curve, roc_auc_score, brier_score_loss, confusion_matrix
from sklearn.feature_selection import RFE
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import optuna
import pandas as pd

- **Importando e separando os dados**

In [59]:

df_train = pd.read_csv('/content/drive/MyDrive/train.csv')
df_test = pd.read_csv('/content/drive/MyDrive/test.csv')
df_lances = pd.read_csv('/content/drive/MyDrive/lances.csv')

In [60]:
# Split entre treino e teste

df_treino = df_train
df_treino_Test = df_test


In [61]:
# Pegando os dados de todos lances dos nossos participantes da tabela treino
df_train_lances = df_lances.merge(df_treino, on='id_participante',how='right')
df_train_lances

,id_lance,id_participante,leilao,mercadoria,dispositivo,tempo,pais,ip,url,conta_pagamento,endereco,resultado
0,2351239.0,6713e820273f591bdff7d26a48ee30c5k4iii,ilxn7,joias,phone61,9.631917e+15,Índia,77.24.141.209,llr13l34u0hwv5b,a3d2de7675556553a5f08e4c88d2c228x2qy8,a3d2de7675556553a5f08e4c88d2c228tfgez,0
1,2351259.0,6713e820273f591bdff7d26a48ee30c5k4iii,c441l,joias,phone542,9.631917e+15,tr,159.104.175.146,m1henrpeyx9olvd,a3d2de7675556553a5f08e4c88d2c228x2qy8,a3d2de7675556553a5f08e4c88d2c228tfgez,0
2,2351287.0,6713e820273f591bdff7d26a48ee30c5k4iii,ox6b6,joias,phone542,9.631917e+15,tr,142.244.73.189,m1henrpeyx9olvd,a3d2de7675556553a5f08e4c88d2c228x2qy8,a3d2de7675556553a5f08e4c88d2c228tfgez,0
3,2351318.0,6713e820273f591bdff7d26a48ee30c5k4iii,5s5ou,joias,phone59,9.631917e+15,ro,165.194.24.251,p7aje1psxt4h12r,a3d2de7675556553a5f08e4c88d2c228x2qy8,a3d2de7675556553a5f08e4c88d2c228tfgez,0
4,2351367.0,6713e820273f591bdff7d26a48ee30c5k4iii,istth,joias,phone150,9.631918e+15,Índia,60.36.70.180,vasstdc27m7nks3,a3d2de7675556553a5f08e4c88d2c228x2qy8,a3d2de7675556553a5f08e4c88d2c228tfgez,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2534336,7650103.0,0318e0a173f7c65db40116b903884c854x258,sm0bp,artigos esportivos,phone128,9.709195e+15,Rússia,39.213.31.142,vasstdc27m7nks3,7bb349beb18645e9d5d41869915f4086zzt7g,c16deb17569fd18c9ca1d5c616caf3ec4fj61,0
2534337,7651290.0,0318e0a173f7c65db40116b903884c854x258,4o0bz,artigos esportivos,phone82,9.709200e+15,de,83.241.148.63,vasstdc27m7nks3,7bb349beb18645e9d5d41869915f4086zzt7g,c16deb17569fd18c9ca1d5c616caf3ec4fj61,0
2534338,7653457.0,0318e0a173f7c65db40116b903884c854x258,9223u,artigos esportivos,phone129,9.709209e+15,Rússia,115.47.187.136,vasstdc27m7nks3,7bb349beb18645e9d5d41869915f4086zzt7g,c16deb17569fd18c9ca1d5c616caf3ec4fj61,0
2534339,7653810.0,0318e0a173f7c65db40116b903884c854x258,qvhei,artigos esportivos,phone4,9.709211e+15,cm,50.0.84.108,vasstdc27m7nks3,7bb349beb18645e9d5d41869915f4086zzt7g,c16deb17569fd18c9ca1d5c616caf3ec4fj61,0


In [62]:
# Pegando os dados de todos lances dos nossos participantes da tabela teste
df_test_lances = df_lances.merge(df_treino_Test, on='id_participante',how='right')
df_test_lances

,id_lance,id_participante,leilao,mercadoria,dispositivo,tempo,pais,ip,url,conta_pagamento,endereco
0,2394993.0,a71f84e476d03bd0b25d45e44eeed9a8efehw,8gshg,joias,phone469,9.632011e+15,my,233.253.145.73,qs3vvoriedvrj0x,a3d2de7675556553a5f08e4c88d2c228h98zm,a3d2de7675556553a5f08e4c88d2c228f0bvv
1,2401931.0,a71f84e476d03bd0b25d45e44eeed9a8efehw,8gshg,joias,phone46,9.632026e+15,my,155.48.112.213,fsikcqh3t003uug,a3d2de7675556553a5f08e4c88d2c228h98zm,a3d2de7675556553a5f08e4c88d2c228f0bvv
2,2406130.0,a71f84e476d03bd0b25d45e44eeed9a8efehw,ca59r,joias,phone46,9.632036e+15,my,28.105.143.62,8ptbt1q05i4h1lu,a3d2de7675556553a5f08e4c88d2c228h98zm,a3d2de7675556553a5f08e4c88d2c228f0bvv
3,2452472.0,a71f84e476d03bd0b25d45e44eeed9a8efehw,8gshg,joias,phone13,9.632152e+15,my,56.49.2.96,vasstdc27m7nks3,a3d2de7675556553a5f08e4c88d2c228h98zm,a3d2de7675556553a5f08e4c88d2c228f0bvv
4,2456348.0,a71f84e476d03bd0b25d45e44eeed9a8efehw,8gshg,joias,phone524,9.632163e+15,my,95.116.232.134,8ptbt1q05i4h1lu,a3d2de7675556553a5f08e4c88d2c228h98zm,a3d2de7675556553a5f08e4c88d2c228f0bvv
...,...,...,...,...,...,...,...,...,...,...,...
536907,1405638.0,6cfd573a82acda92e8207745bca28aa7lqjcm,jefix,equipamentos de escritório,phone1,9.767546e+15,Índia,207.253.4.71,xde77jhswp80wam,a3d2de7675556553a5f08e4c88d2c228o6hvo,1a2d1b624930fe3e707d43e75e92e81e3q8z4
536908,1524990.0,6cfd573a82acda92e8207745bca28aa7lqjcm,jefix,equipamentos de escritório,phone45,9.768242e+15,Índia,60.97.43.31,vasstdc27m7nks3,a3d2de7675556553a5f08e4c88d2c228o6hvo,1a2d1b624930fe3e707d43e75e92e81e3q8z4
536909,1535599.0,6cfd573a82acda92e8207745bca28aa7lqjcm,ygeqt,equipamentos de escritório,phone8,9.768302e+15,my,117.172.183.248,rhxecw236nwuozg,a3d2de7675556553a5f08e4c88d2c228o6hvo,1a2d1b624930fe3e707d43e75e92e81e3q8z4
536910,2037420.0,6cfd573a82acda92e8207745bca28aa7lqjcm,jefix,equipamentos de escritório,phone25,9.770974e+15,Índia,213.225.77.188,vasstdc27m7nks3,a3d2de7675556553a5f08e4c88d2c228o6hvo,1a2d1b624930fe3e707d43e75e92e81e3q8z4


- **Feature Engineering / Pré-processamento dos dados**

Nesta etapa é feita o tratamento das colunas categóricas onde se foi preferido utilizar o método de Target encoder. Além disso testamos alguns insights de features que poderiam ajudar nosso modelo, como por exemplo:




*   De quantos países diferentes esse participante já deu lances?
*   Quantos ips diferentes esse participante deu lances?


*   Time Flag - verifica se o participante tentou dar mais de um lance no mesmo exato momento

*   Qual costuma ser o intervalo de tempo entre um lance e outro desse participante?

*   Quantas lances ele faz para cada troca de ip?
*   Quantos lances ele faz para cada troca de dispositivo?


*   Quantos ips ele usa por país


*   **...**






In [63]:
categorical_features = ['leilao', 'mercadoria', 'dispositivo', 'pais', 'ip', 'url']

In [64]:
pesos =['peso_leilao', 'peso_mercadoria', 'peso_dispositivo', 'peso_pais', 'peso_ip', 'peso_url']

In [65]:
encoder= TargetEncoder(min_samples_leaf=400,smoothing=11.5,cols=categorical_features)

In [66]:
encoder.fit(df_train_lances[categorical_features],df_train_lances['resultado'])

TargetEncoder(cols=['leilao', 'mercadoria', 'dispositivo', 'pais', 'ip', 'url'],
              min_samples_leaf=400, smoothing=11.5)

In [67]:
df_train_lances[pesos] = encoder.transform(df_train_lances[categorical_features])
df_test_lances[pesos] = encoder.transform(df_test_lances[categorical_features])

In [68]:
df_train_lances

,id_lance,id_participante,leilao,mercadoria,dispositivo,tempo,pais,ip,url,conta_pagamento,endereco,resultado,peso_leilao,peso_mercadoria,peso_dispositivo,peso_pais,peso_ip,peso_url
0,2351239.0,6713e820273f591bdff7d26a48ee30c5k4iii,ilxn7,joias,phone61,9.631917e+15,Índia,77.24.141.209,llr13l34u0hwv5b,a3d2de7675556553a5f08e4c88d2c228x2qy8,a3d2de7675556553a5f08e4c88d2c228tfgez,0,0.131551,0.088457,0.108552,0.148849,0.131551,0.131551
1,2351259.0,6713e820273f591bdff7d26a48ee30c5k4iii,c441l,joias,phone542,9.631917e+15,tr,159.104.175.146,m1henrpeyx9olvd,a3d2de7675556553a5f08e4c88d2c228x2qy8,a3d2de7675556553a5f08e4c88d2c228tfgez,0,0.111665,0.088457,0.076699,0.041501,0.131551,0.131551
2,2351287.0,6713e820273f591bdff7d26a48ee30c5k4iii,ox6b6,joias,phone542,9.631917e+15,tr,142.244.73.189,m1henrpeyx9olvd,a3d2de7675556553a5f08e4c88d2c228x2qy8,a3d2de7675556553a5f08e4c88d2c228tfgez,0,0.089146,0.088457,0.076699,0.041501,0.131551,0.131551
3,2351318.0,6713e820273f591bdff7d26a48ee30c5k4iii,5s5ou,joias,phone59,9.631917e+15,ro,165.194.24.251,p7aje1psxt4h12r,a3d2de7675556553a5f08e4c88d2c228x2qy8,a3d2de7675556553a5f08e4c88d2c228tfgez,0,0.000000,0.088457,0.059973,0.069339,0.131551,0.131551
4,2351367.0,6713e820273f591bdff7d26a48ee30c5k4iii,istth,joias,phone150,9.631918e+15,Índia,60.36.70.180,vasstdc27m7nks3,a3d2de7675556553a5f08e4c88d2c228x2qy8,a3d2de7675556553a5f08e4c88d2c228tfgez,0,0.131551,0.088457,0.070431,0.148849,0.131551,0.184389
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2534336,7650103.0,0318e0a173f7c65db40116b903884c854x258,sm0bp,artigos esportivos,phone128,9.709195e+15,Rússia,39.213.31.142,vasstdc27m7nks3,7bb349beb18645e9d5d41869915f4086zzt7g,c16deb17569fd18c9ca1d5c616caf3ec4fj61,0,0.091413,0.180901,0.008904,0.195531,0.131551,0.184389
2534337,7651290.0,0318e0a173f7c65db40116b903884c854x258,4o0bz,artigos esportivos,phone82,9.709200e+15,de,83.241.148.63,vasstdc27m7nks3,7bb349beb18645e9d5d41869915f4086zzt7g,c16deb17569fd18c9ca1d5c616caf3ec4fj61,0,0.131551,0.180901,0.031655,0.690626,0.131551,0.184389
2534338,7653457.0,0318e0a173f7c65db40116b903884c854x258,9223u,artigos esportivos,phone129,9.709209e+15,Rússia,115.47.187.136,vasstdc27m7nks3,7bb349beb18645e9d5d41869915f4086zzt7g,c16deb17569fd18c9ca1d5c616caf3ec4fj61,0,0.003671,0.180901,0.018896,0.195531,0.131551,0.184389
2534339,7653810.0,0318e0a173f7c65db40116b903884c854x258,qvhei,artigos esportivos,phone4,9.709211e+15,cm,50.0.84.108,vasstdc27m7nks3,7bb349beb18645e9d5d41869915f4086zzt7g,c16deb17569fd18c9ca1d5c616caf3ec4fj61,0,0.002412,0.180901,0.050846,0.035556,0.000000,0.184389


In [69]:
# adicionando a coluna time_flag nos dados de treino
grouped_df = df_train_lances.groupby(['id_participante', 'tempo'])

df_train_lances['time_flag'] = grouped_df['tempo'].transform('count')
df_train_lances


,id_lance,id_participante,leilao,mercadoria,dispositivo,tempo,pais,ip,url,conta_pagamento,endereco,resultado,peso_leilao,peso_mercadoria,peso_dispositivo,peso_pais,peso_ip,peso_url,time_flag
0,2351239.0,6713e820273f591bdff7d26a48ee30c5k4iii,ilxn7,joias,phone61,9.631917e+15,Índia,77.24.141.209,llr13l34u0hwv5b,a3d2de7675556553a5f08e4c88d2c228x2qy8,a3d2de7675556553a5f08e4c88d2c228tfgez,0,0.131551,0.088457,0.108552,0.148849,0.131551,0.131551,1.0
1,2351259.0,6713e820273f591bdff7d26a48ee30c5k4iii,c441l,joias,phone542,9.631917e+15,tr,159.104.175.146,m1henrpeyx9olvd,a3d2de7675556553a5f08e4c88d2c228x2qy8,a3d2de7675556553a5f08e4c88d2c228tfgez,0,0.111665,0.088457,0.076699,0.041501,0.131551,0.131551,1.0
2,2351287.0,6713e820273f591bdff7d26a48ee30c5k4iii,ox6b6,joias,phone542,9.631917e+15,tr,142.244.73.189,m1henrpeyx9olvd,a3d2de7675556553a5f08e4c88d2c228x2qy8,a3d2de7675556553a5f08e4c88d2c228tfgez,0,0.089146,0.088457,0.076699,0.041501,0.131551,0.131551,1.0
3,2351318.0,6713e820273f591bdff7d26a48ee30c5k4iii,5s5ou,joias,phone59,9.631917e+15,ro,165.194.24.251,p7aje1psxt4h12r,a3d2de7675556553a5f08e4c88d2c228x2qy8,a3d2de7675556553a5f08e4c88d2c228tfgez,0,0.000000,0.088457,0.059973,0.069339,0.131551,0.131551,1.0
4,2351367.0,6713e820273f591bdff7d26a48ee30c5k4iii,istth,joias,phone150,9.631918e+15,Índia,60.36.70.180,vasstdc27m7nks3,a3d2de7675556553a5f08e4c88d2c228x2qy8,a3d2de7675556553a5f08e4c88d2c228tfgez,0,0.131551,0.088457,0.070431,0.148849,0.131551,0.184389,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2534336,7650103.0,0318e0a173f7c65db40116b903884c854x258,sm0bp,artigos esportivos,phone128,9.709195e+15,Rússia,39.213.31.142,vasstdc27m7nks3,7bb349beb18645e9d5d41869915f4086zzt7g,c16deb17569fd18c9ca1d5c616caf3ec4fj61,0,0.091413,0.180901,0.008904,0.195531,0.131551,0.184389,1.0
2534337,7651290.0,0318e0a173f7c65db40116b903884c854x258,4o0bz,artigos esportivos,phone82,9.709200e+15,de,83.241.148.63,vasstdc27m7nks3,7bb349beb18645e9d5d41869915f4086zzt7g,c16deb17569fd18c9ca1d5c616caf3ec4fj61,0,0.131551,0.180901,0.031655,0.690626,0.131551,0.184389,1.0
2534338,7653457.0,0318e0a173f7c65db40116b903884c854x258,9223u,artigos esportivos,phone129,9.709209e+15,Rússia,115.47.187.136,vasstdc27m7nks3,7bb349beb18645e9d5d41869915f4086zzt7g,c16deb17569fd18c9ca1d5c616caf3ec4fj61,0,0.003671,0.180901,0.018896,0.195531,0.131551,0.184389,1.0
2534339,7653810.0,0318e0a173f7c65db40116b903884c854x258,qvhei,artigos esportivos,phone4,9.709211e+15,cm,50.0.84.108,vasstdc27m7nks3,7bb349beb18645e9d5d41869915f4086zzt7g,c16deb17569fd18c9ca1d5c616caf3ec4fj61,0,0.002412,0.180901,0.050846,0.035556,0.000000,0.184389,1.0


In [70]:
# adicionando a coluna time_flag nos dados de teste
grouped_df2 = df_test_lances.groupby(['id_participante', 'tempo'])


df_test_lances['time_flag'] = grouped_df2['tempo'].transform('count')
df_test_lances['time_flag'].unique()


array([ 1.,  2., nan,  3.,  4.,  5.,  6.,  8.])

In [71]:
#funcao que adiciona a coluna com a diferenca do tempo entre o lance atual e seu ultimo lance

def addtempo(df1):
  df1["tempo"] = df1.groupby("id_participante")["tempo"].transform(np.sort)
  df1["tempo_por_lance"] = df1.groupby("id_participante")["tempo"].diff()
  df1["tempo_por_lance"].fillna(0, inplace=True)
  return df1

addtempo(df_train_lances)
addtempo(df_test_lances)

,id_lance,id_participante,leilao,mercadoria,dispositivo,tempo,pais,ip,url,conta_pagamento,endereco,peso_leilao,peso_mercadoria,peso_dispositivo,peso_pais,peso_ip,peso_url,time_flag,tempo_por_lance
0,2394993.0,a71f84e476d03bd0b25d45e44eeed9a8efehw,8gshg,joias,phone469,9.632011e+15,my,233.253.145.73,qs3vvoriedvrj0x,a3d2de7675556553a5f08e4c88d2c228h98zm,a3d2de7675556553a5f08e4c88d2c228f0bvv,0.131551,0.088457,0.238585,0.119584,0.131551,0.131551,1.0,0.000000e+00
1,2401931.0,a71f84e476d03bd0b25d45e44eeed9a8efehw,8gshg,joias,phone46,9.632026e+15,my,155.48.112.213,fsikcqh3t003uug,a3d2de7675556553a5f08e4c88d2c228h98zm,a3d2de7675556553a5f08e4c88d2c228f0bvv,0.131551,0.088457,0.464956,0.119584,0.131551,0.131551,1.0,1.552632e+10
2,2406130.0,a71f84e476d03bd0b25d45e44eeed9a8efehw,ca59r,joias,phone46,9.632036e+15,my,28.105.143.62,8ptbt1q05i4h1lu,a3d2de7675556553a5f08e4c88d2c228h98zm,a3d2de7675556553a5f08e4c88d2c228f0bvv,0.131551,0.088457,0.464956,0.119584,0.131551,0.131551,1.0,9.684211e+09
3,2452472.0,a71f84e476d03bd0b25d45e44eeed9a8efehw,8gshg,joias,phone13,9.632152e+15,my,56.49.2.96,vasstdc27m7nks3,a3d2de7675556553a5f08e4c88d2c228h98zm,a3d2de7675556553a5f08e4c88d2c228f0bvv,0.131551,0.088457,0.468288,0.119584,0.130557,0.184389,1.0,1.164211e+11
4,2456348.0,a71f84e476d03bd0b25d45e44eeed9a8efehw,8gshg,joias,phone524,9.632163e+15,my,95.116.232.134,8ptbt1q05i4h1lu,a3d2de7675556553a5f08e4c88d2c228h98zm,a3d2de7675556553a5f08e4c88d2c228f0bvv,0.131551,0.088457,0.822840,0.119584,0.131551,0.131551,1.0,1.068421e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
536907,1405638.0,6cfd573a82acda92e8207745bca28aa7lqjcm,jefix,equipamentos de escritório,phone1,9.767546e+15,Índia,207.253.4.71,xde77jhswp80wam,a3d2de7675556553a5f08e4c88d2c228o6hvo,1a2d1b624930fe3e707d43e75e92e81e3q8z4,0.047810,0.059712,0.058496,0.148849,0.115543,0.131551,1.0,2.124579e+12
536908,1524990.0,6cfd573a82acda92e8207745bca28aa7lqjcm,jefix,equipamentos de escritório,phone45,9.768242e+15,Índia,60.97.43.31,vasstdc27m7nks3,a3d2de7675556553a5f08e4c88d2c228o6hvo,1a2d1b624930fe3e707d43e75e92e81e3q8z4,0.047810,0.059712,0.038032,0.148849,0.131551,0.184389,1.0,6.956842e+11
536909,1535599.0,6cfd573a82acda92e8207745bca28aa7lqjcm,ygeqt,equipamentos de escritório,phone8,9.768302e+15,my,117.172.183.248,rhxecw236nwuozg,a3d2de7675556553a5f08e4c88d2c228o6hvo,1a2d1b624930fe3e707d43e75e92e81e3q8z4,0.131551,0.059712,0.056873,0.119584,0.131551,0.131551,1.0,5.973684e+10
536910,2037420.0,6cfd573a82acda92e8207745bca28aa7lqjcm,jefix,equipamentos de escritório,phone25,9.770974e+15,Índia,213.225.77.188,vasstdc27m7nks3,a3d2de7675556553a5f08e4c88d2c228o6hvo,1a2d1b624930fe3e707d43e75e92e81e3q8z4,0.047810,0.059712,0.073556,0.148849,0.097554,0.184389,1.0,2.672632e+12


Aqui foi feita uma conexão com o sqlite para fazer um agrupamento ideal da tabela lances, de forma que guarde as informações corretas a respeito de cada participante.


In [72]:
conn = sqlite3.connect(":memory:")
df_train_lances.to_sql('lances_train',conn,index=False)
df_test_lances.to_sql('lances_test',conn,index=False)

536912

In [73]:
query = """select t.id_participante ,SUM(time_flag),AVG(time_flag),AVG(tempo_por_lance),AVG(peso_ip),SUM(peso_ip),AVG(peso_pais),AVG(peso_leilao),AVG(peso_dispositivo),AVG(peso_mercadoria),AVG(peso_url),COUNT(distinct id_lance) AS n_lances ,COUNT(distinct ip ) as n_ips , COUNT(distinct pais  ) as n_paises,COUNT(distinct dispositivo  )as n_dispositivos , COUNT(distinct mercadoria  )as n_mercadorias , AVG(t.resultado)
from lances_train t

group by t.id_participante
 """
resultado_df= pd.read_sql_query(query,conn)

In [74]:
resultado_df

,id_participante,SUM(time_flag),AVG(time_flag),AVG(tempo_por_lance),AVG(peso_ip),SUM(peso_ip),AVG(peso_pais),AVG(peso_leilao),AVG(peso_dispositivo),AVG(peso_mercadoria),AVG(peso_url),n_lances,n_ips,n_paises,n_dispositivos,n_mercadorias,AVG(t.resultado)
0,001068c415025a009fee375a12cff4fcnht8y,1.0,1.000000,0.000000e+00,0.040472,0.040472,0.053165,0.045073,0.130291,0.088457,0.184389,1,1,1,1,1,0.0
1,01067975436d123f717ee5aba0dd4bbfa0937,543.0,1.000000,2.502666e+10,0.125927,68.378491,0.135389,0.051047,0.068607,0.088457,0.183221,543,398,71,165,1,0.0
2,012441119bcf83b23d4768bb72cea6d6carua,23.0,1.000000,5.377895e+11,0.134301,3.088929,0.120267,0.147452,0.089477,0.130598,0.149930,23,19,10,16,1,0.0
3,01255c2c7c5578c186873422fc00fd7afwk8k,65.0,1.000000,2.042939e+11,0.125015,8.125982,0.139536,0.054514,0.103426,0.088457,0.155125,65,55,5,34,1,0.0
4,01349288df20199905e719f0ff7ee771nwryj,70.0,1.000000,1.882639e+11,0.126182,8.832750,0.142843,0.194807,0.074782,0.130598,0.154196,70,49,4,28,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1404,ff566c87f2babf3dcd67be286e1c6666cxzh1,3.0,1.000000,2.355523e+13,0.131245,0.393735,0.111260,0.028838,0.092189,0.088457,0.131551,3,3,2,3,1,0.0
1405,ff58ffde976a4899dcd89597a7877e18lntgz,885.0,1.004540,1.546442e+10,0.126965,111.856299,0.141708,0.190742,0.073633,0.180901,0.150143,881,614,46,191,1,0.0
1406,ff92ea4abd33ed38601287f0e1d6726dmgx1f,397.0,1.005063,1.944701e+11,0.126654,50.028345,0.031030,0.123715,0.060116,0.180901,0.164056,395,202,1,99,1,0.0
1407,ffc4e2dd2cc08249f299cab46ecbfacfobmr3,22.0,1.000000,3.432876e+12,0.042929,0.944437,0.082532,0.044629,0.075422,0.130598,0.138756,22,18,6,13,1,0.0


In [75]:
query2 = """
select t.id_participante ,SUM(time_flag),AVG(time_flag),AVG(tempo_por_lance),AVG(peso_ip),SUM(peso_ip),AVG(peso_pais),AVG(peso_leilao),AVG(peso_dispositivo),AVG(peso_mercadoria),AVG(peso_url),COUNT(distinct id_lance) AS n_lances ,COUNT(distinct ip ) as n_ips , COUNT(distinct pais  ) as n_paises,COUNT(distinct dispositivo  )as n_dispositivos , COUNT(distinct mercadoria  )as n_mercadorias
from lances_test t

group by t.id_participante
 """
resultado_df2= pd.read_sql_query(query2,conn)

In [76]:
#funcao que adiciona as colunas lances/ip lances/dispositivo   ip/paises ip/dispositivos
def addfeatures(resultado_df):
  resultado_df['lances/ip'] = resultado_df['n_lances']/resultado_df['n_ips']
  resultado_df['lances/dispositivo'] = resultado_df['n_lances']/resultado_df['n_dispositivos']
  resultado_df[['n_lances','n_ips','n_paises','n_dispositivos']] = resultado_df[['n_lances','n_ips','n_paises','n_dispositivos']] .replace(to_replace=0, value=1)
  resultado_df['ip/paises'] = resultado_df['n_ips']/resultado_df['n_paises']
  resultado_df['ip/dispositivo'] = resultado_df['n_ips']/resultado_df['n_dispositivos']
  return resultado_df
addfeatures(resultado_df)
addfeatures(resultado_df2)

,id_participante,SUM(time_flag),AVG(time_flag),AVG(tempo_por_lance),AVG(peso_ip),SUM(peso_ip),AVG(peso_pais),AVG(peso_leilao),AVG(peso_dispositivo),AVG(peso_mercadoria),AVG(peso_url),n_lances,n_ips,n_paises,n_dispositivos,n_mercadorias,lances/ip,lances/dispositivo,ip/paises,ip/dispositivo
0,0030a2dd87ad2733e0873062e4f83954mkj86,1.0,1.000000,0.000000e+00,0.041270,0.041270,0.060363,0.026199,0.039118,0.130598,0.131551,1,1,1,1,1,1.000000,1.000000,1.000000,1.000000
1,00a0517965f18610417ee784a05f494d4dw6e,141.0,1.000000,5.447917e+11,0.110603,15.595022,0.160468,0.082384,0.143266,0.172523,0.138671,141,103,16,67,1,1.368932,2.104478,6.437500,1.537313
2,00cc97158e6f4cb8eac3c0075918b7ffi5k8o,3.0,1.000000,1.832267e+13,0.131551,0.394654,0.243850,0.095388,0.088908,0.059712,0.184389,3,3,2,3,1,1.000000,1.000000,1.500000,1.000000
3,01cda526658455000913950f20cf31a2q6nsf,84.0,1.000000,1.605031e+11,0.132232,11.107446,0.129267,0.136605,0.061303,0.036264,0.158599,84,57,6,45,1,1.473684,1.866667,9.500000,1.266667
4,022ac3a7ce986049d9a4bede83ccf9ddctvg6,2639.0,1.389679,6.189961e+09,0.120869,229.530142,0.445841,0.188764,0.173297,0.036264,0.147550,1899,32,7,18,1,59.343750,105.500000,4.571429,1.777778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599,fe4906ac4c8e2db5ed5a71e8a9d10080u6g7o,1.0,1.000000,0.000000e+00,0.131551,0.131551,0.148849,0.215032,0.051207,0.180901,0.131551,1,1,1,1,1,1.000000,1.000000,1.000000,1.000000
600,ff181014a0d7bb534fb9a35f22e24fe72m1d5,553.0,1.000000,1.396905e+11,0.111050,61.410620,0.230918,0.170639,0.195310,0.059712,0.176267,553,311,7,159,1,1.778135,3.477987,44.428571,1.955975
601,ff3ae9450581aa3de5d2326f70e39d20w8fbl,4.0,1.000000,1.710697e+13,0.131551,0.526205,0.099570,0.122270,0.045817,0.180901,0.131551,4,3,2,3,1,1.333333,1.333333,1.500000,1.000000
602,ffacbed056cbfaa60c1fcf51f0d381bddr3ly,9.0,1.000000,7.844649e+12,0.104203,0.937829,0.121466,0.128122,0.065756,0.180901,0.131551,9,8,3,7,1,1.125000,1.285714,2.666667,1.142857


In [77]:
# descartando a coluna id_participante
tabela_treino= resultado_df.drop(columns='id_participante')
tabela_test= resultado_df2.drop(columns='id_participante')

In [78]:
tabela_treino.corr()

,SUM(time_flag),AVG(time_flag),AVG(tempo_por_lance),AVG(peso_ip),SUM(peso_ip),AVG(peso_pais),AVG(peso_leilao),AVG(peso_dispositivo),AVG(peso_mercadoria),AVG(peso_url),n_lances,n_ips,n_paises,n_dispositivos,n_mercadorias,AVG(t.resultado),lances/ip,lances/dispositivo,ip/paises,ip/dispositivo
SUM(time_flag),1.000000,0.584717,-0.030662,-0.031683,0.834888,-0.037914,-0.028550,-0.014353,0.008447,-0.005452,0.976048,0.780077,0.252164,0.555915,0.014218,0.017629,0.001829,0.058967,0.486381,0.079988
AVG(time_flag),0.584717,1.000000,-0.061541,0.045287,0.579822,0.017251,-0.035016,0.052794,-0.015642,-0.015959,0.603645,0.522234,0.226278,0.441250,0.016922,0.149249,0.151648,0.192496,0.447073,0.079839
AVG(tempo_por_lance),-0.030662,-0.061541,1.000000,-0.068425,-0.056158,0.009326,0.028876,-0.016111,-0.021790,-0.012581,-0.043570,-0.056618,-0.185478,-0.158468,0.041001,-0.094244,-0.023078,-0.037518,-0.083805,-0.041553
AVG(peso_ip),-0.031683,0.045287,-0.068425,1.000000,0.004889,0.284345,0.047699,0.357774,0.003150,0.041242,-0.032019,-0.025962,-0.059310,-0.055887,-0.011581,0.249520,0.102498,0.138371,-0.018377,0.014852
SUM(peso_ip),0.834888,0.579822,-0.056158,0.004889,1.000000,-0.023416,-0.002310,0.015824,-0.001845,0.007699,0.929034,0.968298,0.443128,0.705315,0.024130,0.090724,0.001037,0.074022,0.574966,0.130849
AVG(peso_pais),-0.037914,0.017251,0.009326,0.284345,-0.023416,1.000000,0.160047,0.435597,0.036064,0.066521,-0.036703,-0.014354,-0.066891,-0.079026,0.083208,0.105799,0.159520,0.167766,-0.031807,0.046835
AVG(peso_leilao),-0.028550,-0.035016,0.028876,0.047699,-0.002310,0.160047,1.000000,0.048631,-0.002064,0.073987,-0.023636,0.009956,0.015601,-0.022354,0.016058,0.092089,-0.026831,-0.024456,0.009499,0.001665
AVG(peso_dispositivo),-0.014353,0.052794,-0.016111,0.357774,0.015824,0.435597,0.048631,1.000000,0.175622,0.116928,-0.009473,0.022705,-0.038024,-0.049003,-0.001195,0.185175,0.042266,0.093034,0.019618,0.132965
AVG(peso_mercadoria),0.008447,-0.015642,-0.021790,0.003150,-0.001845,0.036064,-0.002064,0.175622,1.000000,0.098593,0.002361,-0.003577,-0.060178,-0.040678,-0.001339,0.073570,0.000079,0.044283,0.046725,0.173616
AVG(peso_url),-0.005452,-0.015959,-0.012581,0.041242,0.007699,0.066521,0.073987,0.116928,0.098593,1.000000,-0.000920,0.012090,-0.017158,-0.034832,0.059298,0.194250,0.039813,0.048584,0.014861,0.032291


In [79]:
# preenchendo os valorer NaN
tabela_treino.fillna(0,inplace=True)
tabela_test.fillna(0,inplace=True)

- **Desenvolvimento dos modelos**


---

Os modelos inicialmente foram ajustado com a finalidade de minimizar o Brier Score, métrica escolhida para a competição.

In [80]:
# Separando os dados para treino do modelo e validação
X_train = tabela_treino.drop(columns='AVG(t.resultado)')
y_train = tabela_treino['AVG(t.resultado)']

X_test = tabela_test



In [81]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="median")
imputer.fit(X_train)
X_train = imputer.transform(X_train)
X_test = imputer.transform(X_test)

In [82]:
# Definir a métrica F1 para a validação cruzada
f1_scorer = make_scorer(f1_score, average='weighted')

O ótimo desempenho do modelo abaixo que foi separado na validação cruzada após o encoder mostra que para dados conhecidos o modelo vai muito bem , mas é necessário saber generalizar para dados desconhecidos.

In [83]:
modelo = RandomForestClassifier(n_estimators=100, random_state=8290,class_weight={0:1,1:4})
cv_scores = cross_val_score(modelo, X_train, y_train, cv=5, scoring=f1_scorer)


print("Scores da validação cruzada:",  cv_scores)
print("Média da validação cruzada:", cv_scores.mean())

Scores da validação cruzada: [0.9598997  0.94601616 0.94601616 0.94924906 0.97249679]
Média da validação cruzada: 0.9547355729037375


- **Feature Selection**

Visando diminuir a dimensionalidade do modelo e melhorar sua performance, foi escolhida a técnica de feature selection Recursive Feature Elimination (RFE) que basicamente é uma técnica de seleção de características que iterativamente treina um modelo, classifica as características por importância, e remove as menos significativas, repetindo o processo até que o número desejado de características seja alcançado.

In [84]:
selected_features= [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 18]
X_train_selected= X_train[:,selected_features]
X_test_selected= X_test[:,selected_features]

-**Tunagem dos Hiperparâmetros**


---
Para saber os melhores hiperparâmetros, foi feita uma tunagem com a biblioteca Optuna, que faz uma otimização Bayseana dos hiperparâmetros.


Best hyperparameters:  {'n_estimators': 181, 'max_depth': 20, 'min_samples_split': 6, 'min_samples_leaf': 1, 'max_features': 'auto'}  

'n_estimators': 284, 'max_depth': 32, 'min_samples_split': 7, 'min_samples_leaf': 3, 'max_features': 'auto'

In [90]:
# Treinar o modelo final com os melhores hiperparâmetros

best_model = RandomForestClassifier(
    n_estimators= 181,
    max_depth= 20,
    min_samples_split= 6,
    min_samples_leaf=1,
    max_features='auto',
    random_state=9,
    class_weight={0: 1, 1: 11}
)

# Ajustar o modelo final
best_model.fit(X_train_selected, y_train)

# Avaliar o modelo final
y_pred_prob = best_model.predict_proba(X_test_selected)[:, 1]
predicted6= best_model.predict(X_test_selected)




/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


In [91]:
x=resultado_df2
x.reset_index(inplace=True)
x['resultado']=y_pred_prob
x.set_index('id_participante',inplace=True)
x= x[['resultado']]
x

,resultado
id_participante,
0030a2dd87ad2733e0873062e4f83954mkj86,0.012103
00a0517965f18610417ee784a05f494d4dw6e,0.010027
00cc97158e6f4cb8eac3c0075918b7ffi5k8o,0.000000
01cda526658455000913950f20cf31a2q6nsf,0.000000
022ac3a7ce986049d9a4bede83ccf9ddctvg6,0.585133
...,...
fe4906ac4c8e2db5ed5a71e8a9d10080u6g7o,0.000000
ff181014a0d7bb534fb9a35f22e24fe72m1d5,0.136255
ff3ae9450581aa3de5d2326f70e39d20w8fbl,0.000000


In [92]:
# Define the filename
filename = "/content/drive/MyDrive/respostaXXX2.csv"

# Save the DataFrame to a CSV file
x.to_csv(filename)